# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
from sklearn.linear_model import LogisticRegression


C:\Users\arean\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [4]:
# frequency table for prestige and whether or not someone was admitted

pd.crosstab(df['prestige'], df['admit'])

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [5]:
dummy_ranks = df.join(pd.get_dummies(df['prestige'], prefix = 'prestige'))

In [6]:
dummy_ranks.head()

,admit,gre,gpa,prestige,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,3.0,0,0,1,0
1,1,660.0,3.67,3.0,0,0,1,0
2,1,800.0,4.00,1.0,1,0,0,0
3,1,640.0,3.19,4.0,0,0,0,1
4,0,520.0,2.93,4.0,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 4, range of prestige is 1-4. One class variable for each potential class we can have. 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [7]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige_1.0':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


In [8]:
#crosstab prestige 1 admission 
pd.crosstab(handCalc["prestige_1.0"], handCalc["admit"])



admit,0,1
prestige_1.0,,
0,243,93
1,28,33


In [9]:
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab([handCalc['prestige_1.0'],handCalc['prestige_2.0'], handCalc['prestige_3.0'], handCalc['prestige_4.0']], handCalc['admit'])

admit                                                 0   1
prestige_1.0 prestige_2.0 prestige_3.0 prestige_4.0        
0            0            0            1             55  12
                          1            0             93  28
             1            0            0             95  53
1            0            0            0             28  33

#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

the odds are 33:28 = 1.179

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

93:243 = 0.383

#### 3.3 Calculate the odds ratio for getting admitted from #1 ranked school.

Prestige_1: the odds ratio 1.179/0.383 = 3.078

#### 3.4 Write this finding in a sentenance: 

Answer: Our odds ratio is > 1. Being from a #1 ranked school has a great influence in admission to grad school. If our odds ratio was 1, no influence. If odds ratio < 1, least influence. 

#### 3.5 Print the cross tab for prestige_4

In [10]:
pd.crosstab(handCalc["prestige_4.0"], handCalc["admit"])

admit,0,1
prestige_4.0,,
0,216,114
1,55,12


#### 3.6 Calculate the OR for getting admitted when you are from a #4 ranked college. 

(12/55)/(114/216) = 0.413

#### 3.7 Write this finding in a sentence

Answer: The odds ratio < 1, therefore it has a negative influence.

## Part 4. Analysis

In [11]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1.0':])
print data.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


C:\Users\arean\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [12]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [26]:
train_cols = data.ix[:, 'gre':]
del train_cols['prestige_1.0']
train_cols.head()

,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,380.0,3.61,0,1,0,1.0
1,660.0,3.67,0,1,0,1.0
2,800.0,4.00,0,0,0,1.0
3,640.0,3.19,0,0,1,1.0
4,520.0,2.93,0,0,1,1.0


In [14]:
train_cols2 = data.ix[:,'gre':'prestige_4.0']
train_cols2.head()

,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,380.0,3.61,0,0,1,0
1,660.0,3.67,0,0,1,0
2,800.0,4.00,1,0,0,0
3,640.0,3.19,0,0,0,1
4,520.0,2.93,0,0,0,1


#### 4.2 Fit the model

In [15]:
#lm = LogisticRegression()

In [16]:
#lm.fit(train_cols2, data['admit'])

In [17]:
#print lm.intercept_

In [18]:
#print lm.coef_

In [19]:
#logit = sm.Logit(df['admit'], train_cols)

In [20]:
#logit.fit().params

In [27]:
X = train_cols
X = sm.add_constant(X)
y = data['admit']

lm = sm.Logit(y, X)
result = lm.fit()


Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [28]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Wed, 20 Sep 2017   Pseudo R-squ.:                 0.08166
Time:                        15:50:10   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
intercept       -3.8769      1.142     -3.393      0.001      -6.116      -1.638
================================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [29]:
print np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [30]:
conf = result.conf_int()
conf["OR"] = np.exp(result.params)
conf.columns = ['2.5%', '97.5%', 'OR']
conf

,2.5%,97.5%,OR
gre,0.000074,0.004362,1.002221
gpa,0.127619,1.431056,2.180027
prestige_2.0,-1.301337,-0.058936,0.506548
prestige_3.0,-2.014579,-0.662776,0.262192
prestige_4.0,-2.371624,-0.735197,0.211525
intercept,-6.116077,-1.637631,0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: Odds of being admitted to grad school from a #2 ranked college is .48 times more than the odds of being admitted from a college ranked #3 or #4. 

#### 4.6 Interpret the OR of GPA

Answer: Odds of being admitted to grad school is highly influenced by GPA at 2.18 times more than the odds from gre score and college ranking. 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [32]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [33]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [47]:
# recreate the dummy variables
combos.columns = ['gre','gpa', 'prestige', 'intercept']
dummy = combos.join(pd.get_dummies(combos['prestige'], prefix = 'prestige'))
print combos.head()

# keep only what we need for making predictions
col_keep = ['gre','gpa','intercept']
df1 = dummy[col_keep].join(dummy.ix[:,'prestige_1.0':])
del df1['prestige_1.0']
print df1.head()


     gre       gpa  prestige  intercept
0  220.0  2.260000       1.0        1.0
1  220.0  2.260000       2.0        1.0
2  220.0  2.260000       3.0        1.0
3  220.0  2.260000       4.0        1.0
4  220.0  2.453333       1.0        1.0
     gre       gpa  intercept  prestige_2.0  prestige_3.0  prestige_4.0
0  220.0  2.260000        1.0             0             0             0
1  220.0  2.260000        1.0             1             0             0
2  220.0  2.260000        1.0             0             1             0
3  220.0  2.260000        1.0             0             0             1
4  220.0  2.453333        1.0             0             0             0


C:\Users\arean\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


#### 5.2 Make predictions on the enumerated dataset

In [48]:
df1['predict'] = result.predict(df1)
print df1.head()

     gre       gpa  intercept  prestige_2.0  prestige_3.0  prestige_4.0  \
0  220.0  2.260000        1.0             0             0             0   
1  220.0  2.260000        1.0             1             0             0   
2  220.0  2.260000        1.0             0             1             0   
3  220.0  2.260000        1.0             0             0             1   
4  220.0  2.453333        1.0             0             0             0   

    predict  
0  0.827672  
1  0.557380  
2  0.503951  
3  0.090492  
4  0.848114  


#### 5.3 Interpret findings for the last 4 observations

In [49]:
df1[-4:]

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0,predict
396,800.0,4.0,1.0,0,0,0,0.985399
397,800.0,4.0,1.0,1,0,0,0.946509
398,800.0,4.0,1.0,0,1,0,0.934534
399,800.0,4.0,1.0,0,0,1,0.582995


Answer: This model suggests that there is a 95% probability that you will be admitted to grad school if student has perfect GRE and GPA score and from a #2 ranked school. In addition, probability of being admitted with perfect GRE and GPA scores from a #3 or #4 ranked college is 93% and 58% respectively. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.